In [1]:
from data import *
from ml import *

#Model 
from xgboost import XGBClassifier

#AUC 
from sklearn.model_selection import cross_val_score


from sklearn.linear_model import LogisticRegression

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
#Include model score and logistic regression

In [3]:
def model(model,ttype,subset,death,treatment,split,size,missing): 
    #
    #-model: String, model to be trained: 'xgb','elasticnet','' 
    #-ttype: String, determines model to be trained 
    #-subset: String, determines subset: 'Lille','7day','Baseline','Paper'
    #-death: Integer, 30 days or 90 days
    #-treatment: Integer, 1 if treated and else 0
    #-split: String, determines split: 'train-test','mixed','cv'
    #-testsize: Float, Size of test set 
    #-missing: String, provides missingness mechanism: 'none','cc','ipw','mice'
    #
    # MODEL TRAINING
    #
    # Train XGB
    #
    # Read data
    df, X, y, X_train, X_test, y_train, y_test = dataset(subset,death,treatment,split,size,missing)
    #
    if model == 'xgb' :
        random_search = xgb_tuning_cv(X, y)
        training = XGBClassifier(**random_search.best_params_,objective='binary:logistic')
    #
    # Train EBM
    #
    elif model == 'gam':
        random_search = ebm_tuning_cv(X,y)
        training = ExplainableBoostingClassifier(**random_search.best_params_)
    #
    #Model Training
    #  
    if ttype == 'cv':
        #df, X,y = dataset(subset,death,treatment,split,size,missing)
        cv_aucs = cross_val_score(training,X,y,cv=10,scoring='roc_auc')
        scores = cross_val_score(training,X,y,cv=10)
        auc = np.mean(cv_aucs)
        score = np.mean(scores)
        
    else:
        #df, X, y, X_train, X_test, y_train, y_test = dataset(subset,death,treatment,split,size,missing)
        model_fit = model.fit(X_train, y_train)
        score = model_fit.score(X_test, y_test)
        auc = roc_auc_score(y_test, model_fit.predict_proba(X_test)[:, 1])
        
    return round(auc*100,2)
        
        
#Fix assignement of model 

In [4]:
no_mis = []

for d in [90]:
    for x in ['xgb']: #,'gam'
        for y in ['baseline','7day','full']:
            print('death: '+ str(d))
            print()
            print('model: '+ x)
            print('Auc')
            print()
            no_mis.append(model(x,'cv',y,d,1,'cv',0.25,'none'))

death: 90

model: xgb
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


death: 90

model: xgb
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


death: 90

model: xgb
Auc



/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [5]:
no_mis

[72.67, 70.98, 77.8]

In [6]:
no_mis_ut = []

for d in [90]:
    for x in ['xgb']: #,'gam'
        for y in ['baseline','7day','full']:
            #print('death: '+ str(d))
            #print()
            #print('model: '+ x)
            #print('Auc')
            #print()
            no_mis_ut.append(model(x,'cv',y,d,0,'cv',0.25,'none'))

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [7]:
no_mis_ut

[73.59, 70.85, 77.61]

In [8]:
mice_mis_t = []

for d in [90]:
    for x in ['xgb']: #,'gam'
        for y in ['baseline','7day','full']:
            #print('death: '+ str(d))
            #print()
            #print('model: '+ x)
            #print('Auc')
            #print()
            mice_mis_t.append(model(x,'cv',y,d,1,'cv',0.25,'mice'))

mice_mis_t 

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[72.64, 89.5, 77.29]

In [9]:
mice_mis_ut = []

for d in [90]:
    for x in ['xgb']: #,'gam'
        for y in ['baseline','7day','full']:
            #print('death: '+ str(d))
            #print()
            #print('model: '+ x)
            #print('Auc')
            #print()
            mice_mis_ut.append(model(x,'cv',y,d,0,'cv',0.25,'mice'))

mice_mis_ut 

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The 

[73.96, 85.21, 78.97]

In [10]:
ipw_mis_t = []

for d in [90]:
    for x in ['xgb']: #,'gam'
        for y in ['baseline','7day','full']:
            #print('death: '+ str(d))
            #print()
            #print('model: '+ x)
            #print('Auc')
            #print()
            ipw_mis_t.append(model(x,'cv',y,d,1,'cv',0.25,'ipw'))
            
ipw_mis_t

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[67.73, 77.15, 71.85]

In [11]:
ipw_mis_ut = []

for d in [90]:
    for x in ['xgb']: #,'gam'
        for y in ['baseline','7day','full']:
            #print('death: '+ str(d))
            #print()
            #print('model: '+ x)
            #print('Auc')
            #print()
            ipw_mis_ut.append(model(x,'cv',y,d,0,'cv',0.25,'ipw'))
            
ipw_mis_ut

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[79.75, 74.49, 76.11]

In [17]:
cc_mis_t = []

for d in [90]:
    for x in ['xgb']: #,'gam'
        for y in ['baseline','7day','full']:
            #print('death: '+ str(d))
            #print()
            #print('model: '+ x)
            #print('Auc')
            #print()
            cc_mis_t.append(model(x,'cv',y,d,1,'cv',0.25,'cc'))
            
cc_mis_t

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[71.62, 77.47, 70.36]

In [16]:
cc_mis_ut = []

for d in [90]:
    for x in ['xgb']: #,'gam'
        for y in ['baseline','7day','full']:
            #print('death: '+ str(d))
            #print()
            #print('model: '+ x)
            #print('Auc')
            #print()
            cc_mis_ut.append(model(x,'cv',y,d,0,'cv',0.25,'ipw'))
            
cc_mis_ut

/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/lilimatic/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=1000. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[79.89, 73.91, 78.19]